### Import the libraries

In [53]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os
from math import gcd
import numpy as np

In [4]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.absolute()
os.chdir(root_path)
str(root_path)

'/Users/Antoine/data_science_projects/natixis_challenge'

### Load the datasets

In [5]:
df_cockpit = pd.read_csv('./data/cockpit_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_info = pd.read_csv('./data/item_info_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_trend = pd.read_csv('./data/item_trend_20221221.csv').drop('Unnamed: 0', axis=1)
df_mycloud = pd.read_csv('./data/mycloud_20221221.csv').drop('Unnamed: 0', axis=1)
df_tmp_hosts_zabbix = pd.read_csv('./data/tmp_hosts_zabbix_20221221.csv').drop('Unnamed: 0', axis=1)

### Vizualise the datasets

In [6]:
print(df_cockpit.shape)
df_cockpit.head(2)

(5189, 22)


,name_department,iua,name_subfunction,name_function,name_server,model,name_state,name_environment,os,country,...,ram,number_core,number_cpu,type_cpu,electrical_power,server_parent,server_parent2,name_cluster,mycloud,bcloud
0,etu-bgc-etrading-architecture,AV9,Application Server,APPLICATION,SWDCFRAV9779,VIRTUAL_MACHINE,INFUNCTION,Dev,Windows 2016 Standard 10.0.14393,FRANCE,...,16777,2,2,INTEL(R) XEON(R) SILVER 4314 CPU @ 2.40GHZ,NaN,SEPCFRNXFX10808,NaN,NaN,Yes,NaN
1,etu-bgc-etrading-architecture,Y59,Application Server,APPLICATION,SLPAFRETR168,VIRTUAL_MACHINE,INFUNCTION,Prod,Linux 3.10.0-1160.76.1.el7.x86_64 RHEL7.9,FRANCE,...,7629,1,1,INTEL(R) XEON(R) CPU E5-2680 0 @ 2.70GHZ:X86_64,NaN,SEPIFRLINP029,NaN,NaN,No,NaN


In [ ]:
print(df_item_info.shape)
df_item_info.head(2)

(10015, 3)


,itemid,hostid,item_type
0,264670,806,cpu
1,622332,4755,cpu


In [ ]:
print(df_item_trend.shape)
df_item_trend.head(2)

(3866173, 6)


,itemid,clock,value_min,value_avg,value_max,item_type
0,264670,2021-05-27,5.789395,7.956363,50.884403,cpu
1,264670,2021-05-28,5.737689,10.279124,63.836303,cpu


In [52]:
print(df_mycloud.shape)
df_mycloud.head(5)

(35, 5)


,key,NameMyCloud,CPU,RAM,Price
0,1#2,small,1,2,332.15
1,1#4,small_2,1,4,445.30
2,1#6,small_3,1,6,547.50
3,1#8,small_4,1,8,667.95
4,1#12,small_6,1,12,1109.60


In [ ]:
print(df_tmp_hosts_zabbix.shape)
df_tmp_hosts_zabbix.head(2)

(4365, 2)


,hostid,host
0,6,slpdfrora040a
1,800,SWPAFRWNETV44


### Calculate the price of the configuration
We only take into account the mycloud servors (we don't have the catalogue of the other servors).

In [49]:
print(f"{len(df_cockpit[df_cockpit.mycloud=='Yes'].ram.sort_values().unique())} different memory capacities of ram.")

80 different memory capacities of ram.


In [74]:
config = pd.DataFrame(df_cockpit[df_cockpit.mycloud=='Yes'][["iua", "ram", "number_cpu"]].sort_values("ram"))
config["ram"] = config.ram/1000
config.head(2)

,iua,ram,number_cpu
342,GOS,1.76,1
694,Q63,1.76,1


In [75]:
def find_closest(ram, ram_refs):
    for index, ram_ref in enumerate(ram_refs):
        # Initialization
        if index==0:
            ram_final = ram_refs[0]

        # Reccurence
        else:
            if abs(ram-ram_ref) < abs(ram-ram_final):
                ram_final = ram_ref
    return ram_final
find_closest(12380, [1000, 5000, 10000, 15000, 20000])

10000

In [77]:
config["ram_ref"] = config.ram.apply(lambda x: find_closest(x, df_mycloud.RAM))
config.head(3)

,iua,ram,number_cpu,ram_ref
342,GOS,1.760,1,2
694,Q63,1.760,1,2
2323,CDB,1.855,1,2


In [85]:
config2 = config.merge(df_mycloud, how="left", left_on=["ram_ref", "number_cpu"], right_on=["RAM", "CPU"])
config2 = config2.drop(["number_cpu", "ram_ref"], axis=1)
config2.head(3)

,iua,ram,key,NameMyCloud,CPU,RAM,Price
0,GOS,1.760,1#2,small,1,2,332.15
1,Q63,1.760,1#2,small,1,2,332.15
2,CDB,1.855,1#2,small,1,2,332.15


In [86]:
# Calculate the price.
config_price = config2.Price.sum()
f"The total price of the config is {config_price}€."

'The total price of the config is 7726305.83€.'

In [90]:
# Final function
def calculate_price(df_cockpit=df_cockpit, df_mycloud=df_mycloud):
    '''
    Calculate the price of a configuration.

    Args:
        df_cockpit : pandas.DataFrame = table of the configuration
        df_mycloud : pandas.DataFrame = table of the catalogue of prices of mycloud servors

    Return:
        price : float = total price of the configuration
    '''

    def find_closest(ram, ram_refs):
        '''
        Find the closest value to a float in a list of floats.

        Args:
            ram : float = value to find the closest of
            ram_refs : list(float) = reference values
            
        Return: 
            ram_final : float = closets reference value.
        '''
        for index, ram_ref in enumerate(ram_refs):
            # Initialization
            if index==0:
                ram_final = ram_refs[0]

            # Reccurence
            else:
                if abs(ram-ram_ref) < abs(ram-ram_final):
                    ram_final = ram_ref
        return ram_final

    # Extract usable data : mycloud servors only
    config = pd.DataFrame(df_cockpit[df_cockpit.mycloud=='Yes'][["iua", "ram", "number_cpu"]].sort_values("ram"))
    # Convert in GBytes
    config["ram"] = config.ram/1000 

    # Find the servors in the catalogue
    config["ram_ref"] = config.ram.apply(lambda x: find_closest(x, df_mycloud.RAM))

    # Add prices
    config = config.merge(df_mycloud, how="left", left_on=["ram_ref", "number_cpu"], right_on=["RAM", "CPU"])
    config = config.drop(["number_cpu", "ram_ref"], axis=1)

    # Calculate the price.
    config_price = config.Price.sum()

    return config_price

In [91]:
calculate_price()

7726305.83

Function added to src.utils.py.